Data Preprocessing

In [1]:
#installing the natural language tool kit for normalization of the dataset
%pip install nltk

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 6.2 MB/s eta 0:00:00a 0:00:01
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
#imporiting the required dependencies
from collections import defaultdict
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/dipp3r/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dipp3r/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /Users/dipp3r/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/dipp3r/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
%pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 6.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 1.6 MB/s eta 0:00:0000:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.4/393.4 kB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 6.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
data_df = pd.read_csv("../dataset/dataset1.csv",sep=",",header=0,quotechar='"')

In [5]:
# max and minimum length review
max_length_review = data_df["review_text"].iloc[data_df["review_text"].str.len().idxmax()]
print(len(max_length_review))

min_length_review = data_df["review_text"].iloc[data_df["review_text"].str.len().idxmin()]
print(len(min_length_review))

523
9


In [6]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# List of text to tokenize
texts = ["This is the first text.", "And here is the second text.", "Finally, the third text."]

# Tokenize the list of text with max_length=128
tokenized_inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="tf")

# Print the tokenized inputs
print("Input IDs:\n", tokenized_inputs['input_ids'])
print("\nAttention Masks:\n", tokenized_inputs['attention_mask'])


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 48.0/48.0 [00:00<00:00, 93.8kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 3.46MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.04MB/s]
config.json: 100%|██████████| 570/570 [00:00<00:00, 6.15MB/s]


Input IDs:
 tf.Tensor(
[[ 101 2023 2003 1996 2034 3793 1012  102    0]
 [ 101 1998 2182 2003 1996 2117 3793 1012  102]
 [ 101 2633 1010 1996 2353 3793 1012  102    0]], shape=(3, 9), dtype=int32)

Attention Masks:
 tf.Tensor(
[[1 1 1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0]], shape=(3, 9), dtype=int32)


In [7]:
list(data_df["review_text"]) 

['Make it like better with a giant pig bigger than king pig.',
 'These screens are small enough without crowding in more stuff.Tuck it in with the rest of the add ons I never use and please get it out of my face!',
 'Dear Rovio; If you absolutely must continue trying to squeeze dollars from your customers this way, at least have the decency to include an opt out for those like myself who are absolutely NOT going to spend an amount equal to what I paid for the app in the first place just to have some gimmicky and expendable in game item.',
 'App crashes when new power up notice pops up.',
 'It would be nice to have an update that didnt crash the app.',
 'App crashes with new release.',
 'I have had no problems with any of the versions so far.',
 'Please bring back the trajectory line or at least make it to where you can turn it on or off.',
 'This is not worth buying it will crash right from the beginning.',
 'I will say you should make shock so close to the other buttons because I like

In [8]:
import re

def custom_tokenizer(text):
    # Define regular expression pattern to identify contractions
    contraction_pattern = re.compile(r"\b(?:didn't|doesn't|don't|hadn't|hasn't|haven't|isn't|shouldn't|wasn't|weren't|won't|wouldn't)\b|\b(?:I|he|she|it|you|we|they)'(?:d|ll|m|re|s|ve)\b|\b\w+(?:n't)\b")
    words = text.split()
    # Process each token and separate punctuations
    final_tokens = []
    for token in words:
        # Remove leading and trailing spaces
        token = token.strip()
        if token:
            if contraction_pattern.match(token):
                final_tokens.append(token)
            # Tokenize punctuations
            else:
                punctuation_tokens = re.findall(r"\b\w+\b|[\.,!?;]", token)
                final_tokens.extend(punctuation_tokens)
    return final_tokens



In [9]:
from transformers import BertTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from collections import defaultdict
from nltk.corpus import wordnet as wn
import tensorflow as tf

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define NLTK resources
lemmatizer = WordNetLemmatizer()
english_stopwords = set(stopwords.words('english'))
NEGATION_SET = {"aren't", "won't", 'nor',  "doesn't", "weren't", "shan't", 'no', 'hadn', "couldn't", "don't", 'needn', "shouldn't",  'shouldn', 'mustn', "wasn't", 'isn', "needn't", 'ain', "hadn't", "haven't",  "didn't", 'didn', 'couldn', 'weren',  "mustn't", 'hasn', 'mightn', 'wouldn', "wouldn't",  "isn't", 'not', "hasn't",  'wasn',  'aren', 'haven', "mightn't",  'doesn',  'won', 'don'}
english_stopwords = english_stopwords - NEGATION_SET
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def preprocess_comments_bert(reviews, max_length=128):

  # Convert to lowercase
  reviews = [review.lower() for review in reviews]

  # Tokenize using NLTK word tokenizer
  tokenized_reviews = [custom_tokenizer(review) for review in reviews]


  # Remove stopwords and perform lemmatization
  filtered_tokens = [[lemmatizer.lemmatize(word, tag_map[tag[0]]) for word, tag in pos_tag(review) 
                      if (word.isalpha() or word in NEGATION_SET) and word not in english_stopwords] for review in tokenized_reviews]
  
  print(filtered_tokens)
  
  tokenized_inputs = tokenizer(
      [' '.join(review) for review in filtered_tokens],
      add_special_tokens=True,
      padding=True, 
      truncation=True, 
      max_length=max_length, 
      return_tensors="tf"
    )
  
  return tokenized_inputs


In [10]:
review_inputs = preprocess_comments_bert(list(data_df["review_text"]))
review_inputs

[['make', 'like', 'well', 'giant', 'pig', 'big', 'king', 'pig'], ['screen', 'small', 'enough', 'without', 'crowd', 'stuff', 'tuck', 'rest', 'add', 'ons', 'never', 'use', 'please', 'get', 'face'], ['dear', 'rovio', 'absolutely', 'must', 'continue', 'try', 'squeeze', 'dollar', 'customer', 'way', 'least', 'decency', 'include', 'opt', 'like', 'absolutely', 'not', 'go', 'spend', 'amount', 'equal', 'pay', 'app', 'first', 'place', 'gimmicky', 'expendable', 'game', 'item'], ['app', 'crash', 'new', 'power', 'notice', 'pop'], ['would', 'nice', 'update', 'didnt', 'crash', 'app'], ['app', 'crash', 'new', 'release'], ['no', 'problem', 'version', 'far'], ['please', 'bring', 'back', 'trajectory', 'line', 'least', 'make', 'turn'], ['not', 'worth', 'buy', 'crash', 'right', 'beginning'], ['say', 'make', 'shock', 'close', 'button', 'like', 'use', 'bird', 'provide'], ['need', 'change'], ['similarly', 'every', 'single', 'app', 'get', 'update', 'beautiful', 'look', 'one', 'not', 'update'], ['really', 'fix']

{'input_ids': <tf.Tensor: shape=(1234, 62), dtype=int32, numpy=
array([[  101,  2191,  2066, ...,     0,     0,     0],
       [  101,  3898,  2235, ...,     0,     0,     0],
       [  101,  6203, 20996, ...,     0,     0,     0],
       ...,
       [  101,  6904,  2615, ...,     0,     0,     0],
       [  101,  3291,  4030, ...,     0,     0,     0],
       [  101,  6592,  2048, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1234, 62), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1234, 62), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32

In [11]:
#Saving the DataFrame as in serialized format to preserve the properties of the data structures(Tensor)

import pickle

#Define the file path where you want to save the serialized DataFrame
file_path = 'output/serialized_review_dataframe.pkl'

# Step 3: Save the DataFrame using pickle
with open(file_path, 'wb') as f:
    pickle.dump(review_inputs, f)

print("DataFrame saved as a serialized file:", file_path)

DataFrame saved as a serialized file: output/serialized_review_dataframe.pkl
